# This is the notebook for optimizing the model for categorizing our Lego bricks. Categories will be defined by Tom Alphin's Lego Brick Labels (v39).

In [74]:
#Import packages
import os  
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

import math
import random
import shutil

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sn
import inspect
from tqdm import tqdm

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [75]:
print(tf.reduce_sum(tf.random.normal([1000, 1000])))
print(tf.config.list_physical_devices('GPU'))

tf.Tensor(-313.1723, shape=(), dtype=float32)
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


# Comparing Transfer Learning Models

We will be using transfer learning techniques on Keras pre-trained models to create our categorization model. We will first need to test out all of the different Keras Applications and discover which one will best work for our needs.



**Training Dataset:**

We will be using the 447 class training data set referenced in this paper:https://www.iccs-meeting.org/archive/iccs2022/papers/133520608.pdf

**Testing:**

We will be selecting 20 random classes of Lego brick to train our models on. Models will then be compared with each other to determine what will best work for our purposes

**Data Augmentation:**

In order to save time for finding an optimal model, no data augmentation other than resizing our images (without preserving aspect ratio) to match the input of the models will be utilized.

In [76]:
# Create a dictionary of models we will test
model_dict = {}
model_dict["VGG16"] = tf.keras.applications.VGG16
model_dict["VGG19"] = tf.keras.applications.VGG19
model_dict["ResNet101V2"] = tf.keras.applications.ResNet101V2
# model_dict["ResNet152"] = tf.keras.applications.ResNet152
model_dict["ResNet152V2"] = tf.keras.applications.ResNet152V2
model_dict["ResNet50"] = tf.keras.applications.ResNet50
model_dict["ResNet50V2"] = tf.keras.applications.ResNet50V2
model_dict["InceptionResNetV2"] = tf.keras.applications.InceptionResNetV2
model_dict["InceptionV3"] = tf.keras.applications.InceptionV3
model_dict["MobileNet"] = tf.keras.applications.MobileNet
model_dict["MobileNetV2"] = tf.keras.applications.MobileNetV2
# model_dict["MobileNetV3"] = tf.keras.applications.MobileNetV3 <- weird exceptions
model_dict["Xception"] = tf.keras.applications.Xception
model_dict["DenseNet121"] = tf.keras.applications.DenseNet121
model_dict["DenseNet169"] = tf.keras.applications.DenseNet169
model_dict["DenseNet201"] = tf.keras.applications.DenseNet201
model_dict["EfficientNetV2S"] = tf.keras.applications.EfficientNetV2S
model_dict["EfficientNetV2M"] = tf.keras.applications.EfficientNetV2M
model_dict["EfficientNetV2L"] = tf.keras.applications.EfficientNetV2L




In [77]:
# Load in our preprocessed models into a dictionary
preprocessed_models = {}
for model_name, model in tqdm(model_dict.items()):
    print("Processing: " + model_name)
    preprocessed_models[model_name] = model(include_top=False)

  6%|███▌                                                         | 1/17 [00:00<00:02,  5.36it/s]

Processing: VGG16
Processing: VGG19


 12%|███████▏                                                     | 2/17 [00:00<00:03,  4.73it/s]

Processing: ResNet101V2


 18%|██████████▊                                                  | 3/17 [00:01<00:11,  1.25it/s]

...

 88%|████████████████████████████████████████████████████▉       | 15/17 [00:20<00:04,  2.02s/it]

Processing: EfficientNetV2M


 94%|████████████████████████████████████████████████████████▍   | 16/17 [00:24<00:02,  2.37s/it]

Processing: EfficientNetV2L


100%|████████████████████████████████████████████████████████████| 17/17 [00:28<00:00,  1.69s/it]


In [95]:
# Process our models
# (delete the last layer, make all remaining layers untrainable, and add our own trainable layer)

NUM_CLASSES = 5

processed_models = {}
for name, pre_model in preprocessed_models.items():
    print("processing: " + name)
    # Create our empty model (look up sequential vs functional)
    model = keras.models.Sequential()
    
    # Doing include_top = false instead
    # Add all layers from our pre-trained model (last layer already deleted)
    for layer in pre_model.layers[:]:
        model.add(layer)

    # Make all remaining layers untrainable and add our last trainable layer
    for layer in model.layers:
        layer.trainable = False
        
    model.summary()
        
    model.add(layers.Dense(NUM_CLASSES))
    
    # Loss and optimizer functions
    loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True)
    optim = keras.optimizers.Adam(learning_rate=0.001)
    
    # Might try mean average precision for metric because we are categorizing so many classes
    metrics = ["accuracy"]
    
    # Used to do accuracy but someone online recommended MAP
    # https://www.reddit.com/r/learnmachinelearning/comments/xpyv8j/data_set_for_lego_image_classification_800000/
    # metrics = ["accuracy"]

    # Compile our model
    model.compile(optimizer=optim, loss=loss, metrics=metrics)
    
    # Add model to our dict
    processed_models[name] = model
processed_models["VGG16"]

processing: VGG16
Model: "sequential_126"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 block1_conv1 (Conv2D)       (None, None, None, 64)    1792      
                                                                 
 block1_conv2 (Conv2D)       (None, None, None, 64)    36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, None, None, 64)    0         
                                                                 
 block2_conv1 (Conv2D)       (None, None, None, 128)   73856     
                                                                 
 block2_conv2 (Conv2D)       (None, None, None, 128)   147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, None, None, 128)   0         
                                                                 
 block3_conv1 (Conv2D)       (None

ValueError: Exception encountered when calling layer "conv2_block1_out" (type Add).

A merge layer should be called on a list of inputs. Received: inputs=Tensor("Placeholder:0", shape=(None, None, None, 256), dtype=float32) (not a list of tensors)

Call arguments received by layer "conv2_block1_out" (type Add):
  • inputs=tf.Tensor(shape=(None, None, None, 256), dtype=float32)

In [87]:
# Generate tensor image data batches with model specific preprocessing
BASE_DIR = 'images/'
names = ["3003", "3004", "3021", "6091", "852929"]

tf.random.set_seed(1)

ppDict = {}
ppDict["VGG16"] = tf.keras.applications.vgg16.preprocess_input
ppDict["VGG19"] = tf.keras.applications.vgg19.preprocess_input
ppDict["ResNet101V2"] = tf.keras.applications.resnet_v2.preprocess_input
ppDict["ResNet152V2"] = tf.keras.applications.resnet_v2.preprocess_input
ppDict["ResNet50"] = tf.keras.applications.resnet50.preprocess_input
ppDict["ResNet50V2"] = tf.keras.applications.resnet_v2.preprocess_input
ppDict["InceptionResNetV2"] = tf.keras.applications.inception_resnet_v2.preprocess_input
ppDict["InceptionV3"] = tf.keras.applications.inception_v3.preprocess_input
ppDict["MobileNet"] = tf.keras.applications.mobilenet.preprocess_input
ppDict["MobileNetV2"] = tf.keras.applications.mobilenet_v2.preprocess_input
ppDict["Xception"] = tf.keras.applications.xception.preprocess_input
ppDict["DenseNet121"] = tf.keras.applications.densenet.preprocess_input
ppDict["DenseNet169"] = tf.keras.applications.densenet.preprocess_input
ppDict["DenseNet201"] = tf.keras.applications.densenet.preprocess_input
ppDict["EfficientNetV2S"] = tf.keras.applications.efficientnet_v2.preprocess_input
ppDict["EfficientNetV2M"] = tf.keras.applications.efficientnet_v2.preprocess_input
ppDict["EfficientNetV2L"] = tf.keras.applications.efficientnet_v2.preprocess_input

fitModels = {}

for name, model in processed_models.items():
    train_gen = keras.preprocessing.image.ImageDataGenerator(preprocessing_function=ppDict[name])
    valid_gen = keras.preprocessing.image.ImageDataGenerator(preprocessing_function=ppDict[name])
    test_gen = keras.preprocessing.image.ImageDataGenerator(preprocessing_function=ppDict[name])
    
    train_batches = train_gen.flow_from_directory(
        BASE_DIR + 'train',
        target_size=(224, 224),
        class_mode='sparse',
        batch_size=4,
        shuffle=True,
        color_mode="rgb",
        classes=names   
    )

    val_batches = valid_gen.flow_from_directory(
        BASE_DIR + 'val',
        target_size=(224, 224),
        class_mode='sparse',
        batch_size=4,
        shuffle=True,
        color_mode="rgb",
        classes=names
    )

    test_batches = test_gen.flow_from_directory(
        BASE_DIR + 'test',
        target_size=(224, 224),
        class_mode='sparse',
        batch_size=4,
        shuffle=False,
        color_mode="rgb",
        classes=names
    )
    
    epochs = 30
    
    early_stopping = keras.callbacks.EarlyStopping(
        monitor="val_loss",
        patience=5,
        verbose=2
    )

    print("Start training of: " + name)
    model.fit(train_batches, validation_data=val_batches,
              callbacks=[early_stopping],
              epochs=epochs, verbose=2)
    print("Performance of" + name + ": ")
    model.evaluate(test_batches, verbose=2)
    
    
    fitModels[name] = model
    

Found 8463 images belonging to 5 classes.
Found 3528 images belonging to 5 classes.
Found 2115 images belonging to 5 classes.
Start training of: VGG16
Epoch 1/30


InvalidArgumentError: Graph execution error:

Detected at node 'Equal' defined at (most recent call last):
    File "/home/billiam/miniconda3/envs/tf/lib/python3.10/runpy.py", line 196, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "/home/billiam/miniconda3/envs/tf/lib/python3.10/runpy.py", line 86, in _run_code
      exec(code, run_globals)
    File "/home/billiam/miniconda3/envs/tf/lib/python3.10/site-packages/ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "/home/billiam/miniconda3/envs/tf/lib/python3.10/site-packages/traitlets/config/application.py", line 1043, in launch_instance
      app.start()
    File "/home/billiam/miniconda3/envs/tf/lib/python3.10/site-packages/ipykernel/kernelapp.py", line 728, in start
      self.io_loop.start()
    File "/home/billiam/miniconda3/envs/tf/lib/python3.10/site-packages/tornado/platform/asyncio.py", line 215, in start
      self.asyncio_loop.run_forever()
    File "/home/